## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Here we will try to extract more data from the describtions of each job using simple NER, POS & Regex expressions<br>
We will also need to clean the data after we extract it and make sure that there isn't any NANs

**Things we will extract from the dexcribtions:**
- Salaries data
- Mentioned skills
- Requeird languages <sub>*(Natural languages)</sub>
- Requeird years of experience
- Requeird or prefered degree

## <center><strong>Importing the <span style= 'color: #48e0dc'>Packeges</span>

In [9]:
import os
import re
import random
import nltk
import warnings
import matplotlib
import pandas             as pd
import numpy              as np
import seaborn            as sns
import matplotlib.pyplot  as plt

from bs4                  import BeautifulSoup
from google.cloud         import bigquery
from wordcloud            import WordCloud
from IPython.display      import set_matplotlib_formats
from collections          import Counter

In [6]:
%matplotlib inline
plt.rcParams['font.family'] = 'Candara'

warnings.filterwarnings('ignore')

credentials_path :str = '../credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

FONT             :int  = 17
COLORS           :list = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']
NUMERICS         :list = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
                          'uint16', 'uint32', 'uint64']

## <center><strong>Exploring the <span style= 'color: #48e0dc'>Text</span> data
<sub>*And clean the text data.</sub>

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [67]:
project_id :str = 'data-jobs-analysis-db'
dataset_id :str = 'data_jobs_analysis_db'

client   = bigquery.Client(project= project_id)

linkedin_jobs    = client.query(f'SELECT * FROM {dataset_id}.linkedin_jobs;').to_dataframe()
upwork_profiles  = client.query(f'SELECT * FROM {dataset_id}.upwork_profiles;').to_dataframe()
guru_profiles    = client.query(f'SELECT * FROM {dataset_id}.guru_profiles;').to_dataframe()

In [68]:
linkedin_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3455 entries, 0 to 3454
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   jobs_locations      3455 non-null   object
 1   listing_dates       3349 non-null   object
 2   jobs_titles         3455 non-null   object
 3   companies_names     3455 non-null   object
 4   jobs_links          3455 non-null   object
 5   describtion         3455 non-null   object
 6   location_type       3455 non-null   object
 7   employment_type     3455 non-null   object
 8   industry            3414 non-null   object
 9   reqierd_credential  3455 non-null   object
 10  country             3455 non-null   object
 11  job_title           3455 non-null   object
 12  total_jobs          3455 non-null   Int64 
dtypes: Int64(1), object(12)
memory usage: 354.4+ KB


In [73]:
def remove_html_tags(text):
    
    text_with_raw_html : str = BeautifulSoup(text, 'lxml').text
    text               : str = BeautifulSoup(text_with_raw_html, 'lxml').text

    return text

linkedin_jobs['describtion'] = linkedin_jobs['describtion'].apply(remove_html_tags)
# The other dfs describtions doesn't requere this step becuase the are already cleaned

In [124]:
nltk_text.concordance("B.S")

no matches


In [108]:
nltk_text = nltk.Text(nltk.word_tokenize(linkedin_jobs['describtion'][1500]))

In [ ]:
grammar = r"""
    NP: {<DT>?<JJ>*<NN>+}
    VP: {<MD>?<VB.*>+<RB.*>*<IN>?}
    ADJP: {<JJ.*>+<CC>?<JJ.*>+}
    ADVP: {<RB.*>+}
    PP: {<IN><NP>}
    S: {<NP><VP>}
    SBAR: {<IN|WDT|WP|WP\$|WRB><S>}
    CLAUSE: {<S><SBAR>}
    CHUNK: {<NP><VP><ADJP>*<ADVP>*<PP>*}
"""